In [1]:
import pandas as pd
import numpy as np
import pickle
import string
import sklearn
from time import time
import sklearn
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.externals import joblib

In [2]:
newsDF = pd.read_csv("./uci-news-aggregator.csv")

In [259]:
newsDF.sample(n=5)

,TITLE,CATEGORY
419500,alleg “ipad pro” tablet get teas may see earli...,3
313850,formula one driver michael schumach medic file...,2
206921,emg electrod help control prosthet,2
119154,selena gomez blindsid parent decis fire,1
107123,coachella 2014 bu ride festiv alunageorg,1


In [4]:
newsDF = newsDF[['TITLE','CATEGORY']]

In [5]:
newsDF.shape[0]

422419

In [6]:
newsDF.groupby('CATEGORY').describe()

TITLE                                                             \
           count  unique                                                top   
CATEGORY                                                                      
b         115967  111903                                Posted by Imaduddin   
e         152469  146952  The article requested cannot be found! Please ...   
m          45639   43719                                     Share this on:   
t         108344  104733                                      Business Wire   

               
         freq  
CATEGORY       
b          52  
e         130  
m           9  
t          29

In [7]:
newsDF.sample(n=10, replace=True, random_state=99) 

,TITLE,CATEGORY
160385,Netflix: Comcast 'Double Dipping' on Access Fees,t
23587,"Star Wars to start filming in May, includes mu...",e
177337,What to Watch on Monday: Jack (Bauer) and Loui...,e
167080,"Netflix inks deal with Verizon, should improve...",t
239305,Does Babs Bite The Hand That Feeds? Barbra Str...,e
231144,Pictures of the Aluminium Samsung Galaxy S5 Pr...,t
319186,Gary Oldman apologizes for offensive comments ...,e
349252,Another coup for Facebook: LiveRail acquisitio...,b
285893,Google Inc (GOOG) reportedly to launch health ...,t
325556,Alcohol Consumption Responsible for 1 in 10 US...,m


In [8]:
newsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422419 entries, 0 to 422418
Data columns (total 2 columns):
TITLE       422419 non-null object
CATEGORY    422419 non-null object
dtypes: object(2)
memory usage: 6.4+ MB


In [9]:
print("punctuations: ", string.punctuation)

punctuations:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [10]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print("stopwords", stop_words)
porter = PorterStemmer()

stopwords {'of', "doesn't", 'shan', 'an', 'each', 'itself', 'their', "needn't", 'i', 'same', 'for', 'his', 'are', 'them', 'because', 'the', 'again', 'other', 'mightn', "it's", "hasn't", 'when', 'down', 'more', 'but', 'whom', 'her', "mightn't", 'how', 'during', 'at', "didn't", 'yours', 'these', 't', 'shouldn', 'or', 'd', 'very', 'which', 'out', "aren't", 'him', 'don', "that'll", 'you', 'aren', "wasn't", 'he', 'was', 'all', 'with', "wouldn't", 'herself', 'doesn', "haven't", 'there', 'isn', 'while', 'they', 'after', 'until', 'had', 'in', 'wasn', "hadn't", 'those', 'who', 'no', 'above', 'its', 'into', 'now', 'ain', 'am', 'hadn', "shan't", 'such', 'me', 'ourselves', 'ours', 'has', 'too', 'myself', 'than', 'just', 'what', "don't", 'she', 'will', 're', 'our', 'm', 'll', 'we', 'been', 'here', 'your', 'why', 'haven', "you've", 'doing', 'yourself', 'it', 'have', 'yourselves', 'hasn', "won't", 'won', 'only', "shouldn't", 'theirs', "you're", 'most', 'having', 'can', 'does', 'over', 'both', 'my', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dharanidharsamudrala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def preprocess(title):
        
    """ perform lowercase, remove punctuations and then perform stemming and then return string"""
    
    """ convert to lower case"""
    title = title.lower()
    
    """ remove punctuation """
    title = title.translate(str.maketrans("" , "", string.punctuation))
    
    """ split the string to tokens """
    word_data = title.split()
    
    """ remove stopwords """
    new_word_data = []
    for token in word_data:
        if token not in stop_words:
            new_word_data.append(token)
    
    new_title = ""
    
    """ define porter stemmer - perform stemming """
    for word in new_word_data:
        new_title += porter.stem(word) + " "
        
    return new_title

In [12]:
newsDF['TITLE'] = [preprocess(title) for title in newsDF['TITLE']]

In [13]:
features = newsDF['TITLE']

In [14]:
newsDF['CATEGORY'] = newsDF.CATEGORY.map({'b':0,'e':1,'m':2,'t':3})
labels = newsDF['CATEGORY']

In [18]:
#pickle.dump( features, open("your_features.csv", "wb") )
#pickle.dump( labels, open("your_labels.csv", "wb") )

In [266]:
newsDF.sample(n=10, replace=True, random_state=99)

,TITLE,CATEGORY
160385,netflix comcast doubl dip access fee,3
23587,star war start film may includ muchlov r2d2,1
177337,watch monday jack bauer loui ck make triumphan...,1
167080,netflix ink deal verizon improv stream subscrib,3
239305,bab bite hand feed barbra streisand find sex,1
231144,pictur aluminium samsung galaxi s5 prime frust...,3
319186,gari oldman apolog offens comment ahol,1
349252,anoth coup facebook liverail acquisit put near...,0
285893,googl inc goog reportedli launch health data p...,3
325556,alcohol consumpt respons 1 10 us death cdc,2


In [16]:
""" splitting the data into training and testing sets """
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25, random_state=42, shuffle= True)
print('No. of rows in the features_train: {}'.format(features_train.shape[0]))
print('No. of rows in the features_test: {}'.format(features_test.shape[0]))

No. of rows in the features_train: 316814
No. of rows in the features_test: 105605


In [17]:
""" unstructured to structured data"""
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features_train = vectorizer.fit_transform(features_train)
feature_names = vectorizer.get_feature_names()
#print(feature_names[:150])
print(features_train.shape)
print("Non-zero occurences", features_train.nnz)

(316814, 47244)
Non-zero occurences 2192611


In [290]:
print(feature_names[6000:10000])
#pickle.dump(feature_names, open("document_terms.txt", "wb"))

['acdc', 'ace', 'acea', 'acelrx', 'acer', 'acerb', 'aceshowbiz', 'acess', 'acet', 'acevedo', 'achang', 'achangin', 'achc', 'achch', 'achey', 'achi', 'achiev', 'achil', 'achingli', 'achoo', 'acid', 'acidbath', 'acidif', 'acip', 'acircpound349', 'ackman', 'ackmanl', 'ackmanval', 'acknowledg', 'acl', 'aclu', 'acm', 'acmp', 'acn', 'acneremov', 'acog', 'acom', 'acotor', 'acoust', 'acp', 'acquaint', 'acquihir', 'acquir', 'acquisis', 'acquisit', 'acquisition', 'acquisitionjustic', 'acquisiton', 'acquist', 'acquit', 'acquitt', 'acqur', 'acqusit', 'acr', 'acreag', 'acrefeet', 'acria', 'acrimoni', 'acrobat', 'acronym', 'across', 'acrost', 'acroyoga', 'acsi', 'acsm', 'act', 'actavi', 'actiblizz', 'actiev', 'actinium', 'action', 'actionaid', 'actioncamera', 'actionconstancio', 'actionfil', 'actionheavi', 'actionpack', 'actionpacked', 'actionthril', 'activ', 'activis', 'activisionblizzard', 'activison', 'activist', 'activitycontrol', 'activitytrack', 'activs', 'acto', 'actor', 'actorathlet', 'actor

In [19]:
features_test = vectorizer.transform(features_test)

In [20]:
dt_count = features_train.toarray()
dt_count[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [262]:
""" Applying dimensionality reduction and feature selection"""
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features_train, labels_train)
features_train_transformed = selector.transform(features_train)
features_test_transformed  = selector.transform(features_test)
print ("No of features after selection :", features_train_transformed.shape[1])
print("feature scores: ", selector.scores_)
print ('***Features sorted by score:', [feature_names[i] for i in np.argsort(selector.scores_)[::-1]])

No of features after selection : 4725
feature scores:  [4.07 3.22 0.88 ... 1.47 1.47 1.47]
***Features sorted by score: ['ebola', 'googl', 'appl', 'studi', 'samsung', 'cancer', 'microsoft', 'mer', 'stock', 'facebook', 'viru', 'outbreak', 'health', 'galaxi', 'fda', 'star', 'android', 'us', 'kardashian', 'kim', 'bank', 'iphon', 'season', 'launch', 'patient', 'movi', 'drug', 's5', 'ecigarett', 'risk', 'rate', 'diseas', 'cdc', 'autism', 'case', 'throne', 'alzheim', 'medic', 'profit', 'glass', 'film', 'china', 'wed', 'market', 'spread', 'price', 'kany', 'oil', 'window', 'nasa', 'app', 'xbox', 'guinea', 'trailer', 'miley', 'fed', 'user', 'euro', 'cyru', 'saudi', 'award', 'climat', 'earn', 'ecb', 'rise', 'test', 'dollar', 'death', 'smartphon', 'chri', 'billion', 'bieber', 'growth', 'heart', 'heartble', 'vaccin', 'link', 'obes', 'justin', 'fall', 'episod', 'doctor', 'phone', 'recal', 'io', 'diabet', 'brain', 'game', 'mental', 'sale', 'video', 'nile', 'west', 'review', 'htc', 'one', 'trade', 'd

In [22]:
""" Grid search CV for models"""
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [23]:
params = [

    {
        'alpha': [0.001, 0.01, 0.1, 0.5, 1, 10]
    },

    {
         'multi_class' : ('multinomial', 'ovr'),
         'solver' : ('newton-cg', 'sag', 'saga'),
         'C': [0.01, 0.1, 1, 10, 100]
    },

    {
         'criterion' : ('gini', 'entropy'),
         'n_estimators': [10, 50, 100, 150, 200],
         'max_depth' : range(1,10),
         'max_features' : ('sqrt', 'log2'),
    }
]

In [133]:
models = []
models.append(('MNB', MultinomialNB()))
models.append(('LR', LogisticRegression(class_weight = 'balanced')))
models.append(('RF', RandomForestClassifier()))

In [134]:
len(models)

3

In [135]:
clfWithBestParameters = []

In [136]:
i = 0
for name, clf in models:

    print ("\nFitting the classifier ", name, " to transformed dataset")
    t0 = time()
    classifier = GridSearchCV(clf, params[i])
    classifier = classifier.fit(features_train_transformed, labels_train)
    print ("done in : ", round(time() - t0,3), "s")
    print ("best parameters selected for transformed features : \n", classifier.best_params_)
    parameters = classifier.best_estimator_
    clfWithBestParameters.append(parameters)

    i += 1


Fitting the classifier  MNB  to transformed dataset


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


done in :  1.812 s
best parameters selected for transformed features : 
 {'alpha': 0.01}

Fitting the classifier  LR  to transformed dataset


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did

done in :  957.931 s
best parameters selected for transformed features : 
 {'C': 10, 'multi_class': 'ovr', 'solver': 'sag'}

Fitting the classifier  RF  to transformed dataset


/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


done in :  2555.375 s
best parameters selected for transformed features : 
 {'criterion': 'gini', 'max_depth': 9, 'max_features': 'sqrt', 'n_estimators': 200}


In [144]:
len(clfWithBestParameters)

3

In [138]:
parameters

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [143]:
print(clfWithBestParameters[0])
print(clfWithBestParameters[1])
print(clfWithBestParameters[2])

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
          solver='sag', tol=0.0001, verbose=0, warm_start=False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [145]:
i = 0
highestAcc = 0
for name, clf in models:

    # estimate the accuracy of a model on the dataset by splitting the data, fitting a
    # model and computing the score 10 consecutive times (with 10 different splits each time)
    skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)

    # finding accuracy with transformed features
    classifier = clfWithBestParameters[i]
    acc = cross_val_score(classifier, features_train_transformed, labels_train, cv = skf, scoring='accuracy')
    print ("accuracy for ", name, " with transformed features : ", acc.mean())

    if highestAcc < acc.mean():
        highestAcc = acc.mean()
        bestModel = clone(classifier)

    i += 1


print ("So the most accurate model is : ", bestModel)

accuracy for  MNB  with transformed features :  0.9121156432807258


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


accuracy for  LR  with transformed features :  0.9300283590026408
accuracy for  RF  with transformed features :  0.5137052296065712
So the most accurate model is :  LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
          solver='sag', tol=0.0001, verbose=0, warm_start=False)


In [146]:
bestModel.fit(features_train, labels_train)
predictions = bestModel.predict(features_test)
print("Testing accuracy: ", accuracy_score(labels_test, predictions))
print("Confusion matrix: \n", confusion_matrix(labels_test, predictions))
#print(classification_report(labels_test, predictions)) 

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Testing accuracy:  0.9466029070593248
Confusion matrix: 
 [[27028   305   389  1395]
 [  462 36943   225   358]
 [  322   132 10739   126]
 [ 1410   311   204 25256]]


In [147]:
filename = 'bestmodel.sav'
pickle.dump(bestModel, open(filename, 'wb'))

In [148]:
loaded_model = pickle.load(open(filename, 'rb'))

In [149]:
loaded_model

LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
          solver='sag', tol=0.0001, verbose=0, warm_start=False)

**Testing the model with real data**

In [278]:
real_data = pd.read_csv("./testing.csv")

In [279]:
real_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 2 columns):
TITLE       496 non-null object
CATEGORY    496 non-null object
dtypes: object(2)
memory usage: 7.8+ KB


In [280]:
real_data['TITLE'] = [preprocess(title) for title in real_data['TITLE']]

In [281]:
test_features = real_data['TITLE']

In [282]:
real_data['CATEGORY'] = real_data.CATEGORY.map({'b':0,'e':1,'m':2,'t':3})
test_labels = real_data['CATEGORY']

In [283]:
test_features = vectorizer.transform(test_features)

In [284]:
test_predictions = loaded_model.predict(test_features)

In [285]:
print("Testing accuracy: ", accuracy_score(test_labels, test_predictions))
print("Confusion matrix: \n", confusion_matrix(test_labels, test_predictions))

Testing accuracy:  0.7338709677419355
Confusion matrix: 
 [[ 42   9   4  15]
 [ 16 157   9  13]
 [ 18  27 141   9]
 [  7   3   2  24]]


In [286]:
real_data['PREDICTED_CATEGORY'] = test_predictions

In [287]:
real_data.sample(n=10)

,TITLE,CATEGORY,PREDICTED_CATEGORY
446,remot nt resid launch defam suit channel seven...,1,1
357,sydney man contract hiv despit take prevent pill,2,2
35,mark zuckerberg plan split facebook two compani,0,3
167,fda expand blood pressur drug recal fifth time...,2,2
320,milliondollar review nt health blowout kept wrap,2,0
306,gift mother son grandmoth explain decis act su...,2,1
88,easter egg taylor swift video hint new album t...,1,1
55,buy siriu xm stock dip,0,0
424,avicii collabor releas final album first song,1,1
214,may 2019 3 thing stargaz look night sky,3,3


In [288]:
real_data.to_csv('real_data_testing.csv')